In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-1]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-1]

scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

Y = np.array(Y)
y = np.array(y)

X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))

Using TensorFlow backend.


In [2]:
cnn = Sequential()
cnn.add(Convolution1D(64, 3, border_mode="same",activation="relu",input_shape=(41, 1)))
cnn.add(MaxPooling1D(pool_length=(2)))
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))
cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
cnn.summary()
a=[]
folds=5

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 41, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 164,353
Trainable params: 164,353
Non-trainable params: 0
________________________________________________

/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", input_shape=(41, 1), padding="same")`
  
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
for i in range(folds):
    print("Training on Fold: ",i+1)
    XTrain, XVal, yTrain, yVal = train_test_split(X, Y, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
    cnn.fit(XTrain, yTrain, nb_epoch=200, validation_data=(XVal, yVal))
    scores = cnn.evaluate(XVal, yVal, verbose=0)
    print("%s: %.2f%%" % (cnn.metrics_names[1], scores[1]*100))
    a.append(scores[1] * 100)
    print("======="*12, end="\n\n\n")

Training on Fold:  1


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 140272 samples, validate on 35069 samples
Epoch 1/200
140272/140272 [==============================] - 26s 188us/step - loss: 0.3975 - accuracy: 0.7954 - val_loss: 0.3744 - val_accuracy: 0.8110
Epoch 2/200
140272/140272 [==============================] - 25s 181us/step - loss: 0.3674 - accuracy: 0.8200 - val_loss: 0.3558 - val_accuracy: 0.8216
Epoch 3/200
140272/140272 [==============================] - 25s 179us/step - loss: 0.3595 - accuracy: 0.8235 - val_loss: 0.3544 - val_accuracy: 0.8223
Epoch 4/200
140272/140272 [==============================] - 25s 181us/step - loss: 0.3556 - accuracy: 0.8253 - val_loss: 0.3456 - val_accuracy: 0.8266
Epoch 5/200
140272/140272 [==============================] - 25s 179us/step - loss: 0.3527 - accuracy: 0.8270 - val_loss: 0.3442 - val_accuracy: 0.8262
Epoch 6/200
140272/140272 [==============================] - 26s 186us/step - loss: 0.3504 - accuracy: 0.8273 - val_loss: 0.3453 - val_accuracy: 0.8282
Epoch 7/200
140272/140272 [==========

140272/140272 [==============================] - 41s 294us/step - loss: 0.2532 - accuracy: 0.8831 - val_loss: 0.2493 - val_accuracy: 0.8797
Epoch 108/200
140272/140272 [==============================] - 41s 291us/step - loss: 0.2531 - accuracy: 0.8823 - val_loss: 0.2580 - val_accuracy: 0.8744
Epoch 109/200
140272/140272 [==============================] - 41s 292us/step - loss: 0.2529 - accuracy: 0.8827 - val_loss: 0.2411 - val_accuracy: 0.8862
Epoch 110/200
140272/140272 [==============================] - 41s 291us/step - loss: 0.2529 - accuracy: 0.8831 - val_loss: 0.2453 - val_accuracy: 0.8807
Epoch 111/200
140272/140272 [==============================] - 41s 292us/step - loss: 0.2519 - accuracy: 0.8833 - val_loss: 0.2485 - val_accuracy: 0.8833
Epoch 112/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2531 - accuracy: 0.8823 - val_loss: 0.2429 - val_accuracy: 0.8855
Epoch 113/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.25

140272/140272 [==============================] - 41s 293us/step - loss: 0.2488 - accuracy: 0.8842 - val_loss: 0.2489 - val_accuracy: 0.8903
Epoch 13/200
140272/140272 [==============================] - 41s 293us/step - loss: 0.2484 - accuracy: 0.8849 - val_loss: 0.2371 - val_accuracy: 0.8916
Epoch 14/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2493 - accuracy: 0.8848 - val_loss: 0.2403 - val_accuracy: 0.8904
Epoch 15/200
140272/140272 [==============================] - 42s 297us/step - loss: 0.2486 - accuracy: 0.8847 - val_loss: 0.2382 - val_accuracy: 0.8910
Epoch 16/200
140272/140272 [==============================] - 41s 293us/step - loss: 0.2497 - accuracy: 0.8842 - val_loss: 0.2381 - val_accuracy: 0.8930
Epoch 17/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.2487 - accuracy: 0.8850 - val_loss: 0.2401 - val_accuracy: 0.8890
Epoch 18/200
140272/140272 [==============================] - 41s 296us/step - loss: 0.2485 - a

...

140272/140272 [==============================] - 42s 296us/step - loss: 0.2466 - accuracy: 0.8862 - val_loss: 0.2357 - val_accuracy: 0.8917
Epoch 130/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2457 - accuracy: 0.8860 - val_loss: 0.2400 - val_accuracy: 0.8919
Epoch 131/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2467 - accuracy: 0.8861 - val_loss: 0.2374 - val_accuracy: 0.8907
Epoch 132/200
140272/140272 [==============================] - 41s 293us/step - loss: 0.2471 - accuracy: 0.8859 - val_loss: 0.2364 - val_accuracy: 0.8881
Epoch 133/200
140272/140272 [==============================] - 42s 296us/step - loss: 0.2468 - accuracy: 0.8857 - val_loss: 0.2384 - val_accuracy: 0.8898
Epoch 134/200
140272/140272 [==============================] - 42s 297us/step - loss: 0.2463 - accuracy: 0.8852 - val_loss: 0.2346 - val_accuracy: 0.8918
Epoch 135/200
140272/140272 [==============================] - 41s 295us/step - loss: 0.24

140272/140272 [==============================] - 41s 292us/step - loss: 0.2461 - accuracy: 0.8866 - val_loss: 0.2435 - val_accuracy: 0.8831
Epoch 35/200
140272/140272 [==============================] - 41s 293us/step - loss: 0.2453 - accuracy: 0.8868 - val_loss: 0.2448 - val_accuracy: 0.8829
Epoch 36/200
140272/140272 [==============================] - 41s 292us/step - loss: 0.2458 - accuracy: 0.8861 - val_loss: 0.2417 - val_accuracy: 0.8865
Epoch 37/200
140272/140272 [==============================] - 41s 292us/step - loss: 0.2452 - accuracy: 0.8868 - val_loss: 0.2482 - val_accuracy: 0.8811
Epoch 38/200
140272/140272 [==============================] - 41s 292us/step - loss: 0.2461 - accuracy: 0.8863 - val_loss: 0.2416 - val_accuracy: 0.8880
Epoch 39/200
140272/140272 [==============================] - 42s 296us/step - loss: 0.2460 - accuracy: 0.8862 - val_loss: 0.2441 - val_accuracy: 0.8862
Epoch 40/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2455 - a

140272/140272 [==============================] - 41s 294us/step - loss: 0.2457 - accuracy: 0.8865 - val_loss: 0.2464 - val_accuracy: 0.8858
Epoch 141/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2454 - accuracy: 0.8866 - val_loss: 0.2452 - val_accuracy: 0.8855
Epoch 142/200
140272/140272 [==============================] - 42s 298us/step - loss: 0.2457 - accuracy: 0.8866 - val_loss: 0.2435 - val_accuracy: 0.8877
Epoch 143/200
140272/140272 [==============================] - 42s 297us/step - loss: 0.2467 - accuracy: 0.8863 - val_loss: 0.2429 - val_accuracy: 0.8857
Epoch 144/200
140272/140272 [==============================] - 42s 297us/step - loss: 0.2464 - accuracy: 0.8862 - val_loss: 0.2400 - val_accuracy: 0.8878
Epoch 145/200
140272/140272 [==============================] - 42s 298us/step - loss: 0.2463 - accuracy: 0.8863 - val_loss: 0.2410 - val_accuracy: 0.8879
Epoch 146/200
140272/140272 [==============================] - 41s 295us/step - loss: 0.24

140272/140272 [==============================] - 41s 294us/step - loss: 0.2574 - accuracy: 0.8769 - val_loss: 0.2486 - val_accuracy: 0.8788
Epoch 46/200
140272/140272 [==============================] - 41s 295us/step - loss: 0.2577 - accuracy: 0.8770 - val_loss: 0.2495 - val_accuracy: 0.8744
Epoch 47/200
140272/140272 [==============================] - 41s 294us/step - loss: 0.2575 - accuracy: 0.8770 - val_loss: 0.2503 - val_accuracy: 0.8753
Epoch 48/200
140272/140272 [==============================] - 41s 295us/step - loss: 0.2570 - accuracy: 0.8771 - val_loss: 0.2452 - val_accuracy: 0.8777
Epoch 49/200
140272/140272 [==============================] - 42s 297us/step - loss: 0.2563 - accuracy: 0.8774 - val_loss: 0.2645 - val_accuracy: 0.8672
Epoch 50/200
140272/140272 [==============================] - 41s 295us/step - loss: 0.2558 - accuracy: 0.8782 - val_loss: 0.2477 - val_accuracy: 0.8791
Epoch 51/200
140272/140272 [==============================] - 41s 289us/step - loss: 0.2577 - a

140272/140272 [==============================] - 42s 296us/step - loss: 0.2550 - accuracy: 0.8783 - val_loss: 0.2478 - val_accuracy: 0.8758
Epoch 152/200
140272/140272 [==============================] - 41s 293us/step - loss: 0.2556 - accuracy: 0.8780 - val_loss: 0.2446 - val_accuracy: 0.8821
Epoch 153/200
140272/140272 [==============================] - 41s 293us/step - loss: 0.2560 - accuracy: 0.8787 - val_loss: 0.2494 - val_accuracy: 0.8770
Epoch 154/200
140272/140272 [==============================] - 42s 300us/step - loss: 0.2556 - accuracy: 0.8785 - val_loss: 0.2463 - val_accuracy: 0.8811
Epoch 155/200
140272/140272 [==============================] - 41s 291us/step - loss: 0.2552 - accuracy: 0.8788 - val_loss: 0.2470 - val_accuracy: 0.8766
Epoch 156/200
140272/140272 [==============================] - 40s 287us/step - loss: 0.2542 - accuracy: 0.8783 - val_loss: 0.2469 - val_accuracy: 0.8783
Epoch 157/200
140272/140272 [==============================] - 41s 292us/step - loss: 0.25

In [5]:
print (a)

[88.97316455841064, 88.34012746810913, 89.16707038879395, 87.58447766304016, 86.87729835510254]
